In [0]:
CATALOG = "workspace_webcreative1"
BRONZE_SCHEMA = "bronze"
SILVER_SCHEMA = "silver"
GOLD_SCHEMA = "gold"

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {BRONZE_SCHEMA}")

In [0]:
# --- Catalog / schemas ---
CATALOG = "workspace_webcreative1"
BRONZE_SCHEMA = "bronze"
SILVER_SCHEMA = "silver"
GOLD_SCHEMA   = "gold"

# --- Storage paths (RAW) ---
ACCOUNT = "stdbxfinalwebcreative01"
FQDN = f"{ACCOUNT}.dfs.core.windows.net"
RAW_CONTAINER = "raw"

AUTOMOBILE_CSV = f"abfss://{RAW_CONTAINER}@{FQDN}/automobile/Automobile.csv"
ECOMMERCE_CSV  = f"abfss://{RAW_CONTAINER}@{FQDN}/ecommerce/ecommerce.csv"

# --- Tables names ---
BRONZE_AUTO = f"{CATALOG}.{BRONZE_SCHEMA}.automobile_raw"
BRONZE_ECOM = f"{CATALOG}.{BRONZE_SCHEMA}.ecommerce_raw"

SILVER_AUTO = f"{CATALOG}.{SILVER_SCHEMA}.automobile_clean"
SILVER_ECOM = f"{CATALOG}.{SILVER_SCHEMA}.ecommerce_clean"

GOLD_AUTO_KPI = f"{CATALOG}.{GOLD_SCHEMA}.automobile_kpis"
GOLD_ECOM_KPI = f"{CATALOG}.{GOLD_SCHEMA}.ecommerce_kpis"

In [0]:
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {BRONZE_SCHEMA}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SILVER_SCHEMA}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {GOLD_SCHEMA}")

In [0]:
def sniff_delimiter(path: str) -> str:
    # lee una línea como texto con dbutils (rápido)
    head = dbutils.fs.head(path, 2048)
    # heurística simple
    return ";" if head.count(";") > head.count(",") else ","

In [0]:
ACCOUNT = "stdbxfinalwebcreative01"
FQDN = f"{ACCOUNT}.dfs.core.windows.net"

keys = [
  "fs.azure.account.key",
  "spark.hadoop.fs.azure.account.key",
  f"fs.azure.account.key.{FQDN}",
  f"spark.hadoop.fs.azure.account.key.{FQDN}",
]

for k in keys:
    try:
        spark.conf.unset(k)
    except:
        pass

hconf = spark._jsc.hadoopConfiguration()
for k in keys:
    try:
        hconf.unset(k)
    except:
        pass

print("Hadoop/Spark key configs cleaned")

In [0]:
from pyspark.sql.functions import current_timestamp, lit

def read_csv_with_fallback(path):
    # 1) intenta con coma
    try:
        return (spark.read.format("csv")
                .option("header","true")
                .option("inferSchema","true")
                .option("sep", ",")
                .option("multiLine","true")
                .option("escape","\"")
                .load(path))
    except Exception as e1:
        # 2) intenta con punto y coma
        return (spark.read.format("csv")
                .option("header","true")
                .option("inferSchema","true")
                .option("sep", ";")
                .option("multiLine","true")
                .option("escape","\"")
                .load(path))

df_auto = read_csv_with_fallback(AUTOMOBILE_CSV) \
    .withColumn("_ingest_ts", current_timestamp()) \
    .withColumn("_source_file", lit(AUTOMOBILE_CSV))

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
def read_csv_with_fallback(path):
    try:
        return (spark.read.format("csv")
                .option("header","true")
                .option("inferSchema","true")
                .option("sep", ",")
                .option("multiLine","true")
                .option("escape","\"")
                .load(path))
    except:
        return (spark.read.format("csv")
                .option("header","true")
                .option("inferSchema","true")
                .option("sep", ";")
                .option("multiLine","true")
                .option("escape","\"")
                .load(path))

df_ecom = read_csv_with_fallback(ECOMMERCE_CSV)
display(df_ecom.limit(10))